# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
previsao_de_renda = 'https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2013%20Regress%C3%A3o%20II/database/previsao_de_renda.csv'
df = pd.read_csv(previsao_de_renda, index_col=0)
df

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usage: 1

### Limpeza dos dados

In [4]:
df = df.dropna()
df

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,2016-03-01,11477,M,True,False,0,Empresário,Superior incompleto,Casado,Casa,32,9.849315,2.0,1592.57
14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [5]:
df = df.drop_duplicates()
df

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,2016-03-01,11477,M,True,False,0,Empresário,Superior incompleto,Casado,Casa,32,9.849315,2.0,1592.57
14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [6]:
df.reset_index(drop=True, inplace=True)

#### Analisando as categorias em cada variável

In [7]:
df.tipo_renda.value_counts()

tipo_renda
Assalariado         7444
Empresário          3391
Servidor público    1237
Bolsista               9
Pensionista            9
Name: count, dtype: int64

In [8]:
df.educacao.value_counts()

educacao
Secundário             6848
Superior completo      4568
Superior incompleto     545
Primário                103
Pós graduação            26
Name: count, dtype: int64

In [9]:
df.estado_civil.value_counts()

estado_civil
Casado      8647
Solteiro    1499
União        906
Separado     723
Viúvo        315
Name: count, dtype: int64

In [10]:
df.tipo_residencia.value_counts()

tipo_residencia
Casa             10772
Com os pais        658
Governamental      350
Aluguel            177
Estúdio             72
Comunitário         61
Name: count, dtype: int64

In [11]:
df.qt_pessoas_residencia.value_counts()

qt_pessoas_residencia
2.0     6287
3.0     2429
1.0     1918
4.0     1257
5.0      174
6.0       18
9.0        4
15.0       2
7.0        1
Name: count, dtype: int64

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [12]:
y, X = patsy.dmatrices(formula_like='''
                                    np.log(renda) ~ sexo 
                                                    + posse_de_veiculo 
                                                    + posse_de_imovel 
                                                    + C(qtd_filhos, Treatment(0))
                                                    + C(tipo_renda, Treatment('Assalariado'))
                                                    + C(educacao, Treatment('Secundário')) 
                                                    + C(estado_civil, Treatment('Casado')) 
                                                    + C(tipo_residencia, Treatment('Casa')) 
                                                    + idade 
                                                    + tempo_emprego 
                                                    + C(qt_pessoas_residencia, Treatment(2))
                                    ''', data=df)

sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     198.0
Date:                Sun, 26 May 2024   Prob (F-statistic):               0.00
Time:                        13:55:04   Log-Likelihood:                -13205.
No. Observations:               12090   AIC:                         2.648e+04
Df Residuals:                   12055   BIC:                         2.674e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.0875      0.039    181.152      0.000       7.011       7.164
sexo[T.M]                                                       0.7891      0.015     52.965      0.000       0.760       0.818
posse_de_veiculo[T.True]                                        0.0419      0.014      2.920      0.004       0.014       0.070
posse_de_imovel[T.True]                                         0.0839      0.014      5.895      0.000       0.056       0.112
C(qtd_filhos, Treatment(0))[T.1]                               -0.2486      0.112     -2.222      0.026      -0.468      -0.029
C(qtd_filhos, Treatment(0))[T.2]                               -0.5698      0.223     -2.556      0.011      -1.007      -0.133
C(qtd_filhos, Treatment(0))[T.3]                               -0.5573      0.487     -1.145      0.252      -1.512       0.397
C(qtd_filhos, Treatment(0))[T.4]                                0.2070      0.429      0.482      0.629      -0.634       1.048
C(qtd_filhos, Treatment(0))[T.5]                                0.1296      0.411      0.316      0.752      -0.676       0.935
C(qtd_filhos, Treatment(0))[T.7]                                0.0597      0.181      0.330      0.741      -0.295       0.414
C(qtd_filhos, Treatment(0))[T.14]                              -0.4643      0.262     -1.775      0.076      -0.977       0.049
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2277      0.241      0.943      0.346      -0.246       0.701
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1541      0.015     10.144      0.000       0.124       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.2841      0.243     -1.169      0.243      -0.761       0.192
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0555      0.023      2.461      0.014       0.011       0.100
C(educacao, Treatment('Secundário'))[T.Primário]                0.0120      0.072      0.167      0.867      -0.129       0.153
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1180      0.143      0.828      0.408      -0.161       0.397
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1068      0.014      7.559      0.000       0.079       0.135
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0293      0.033     -0.897      0.370      -0.093       0.035
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3212      0.112      2.858      0.004       0.101       0.541
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2522      0.110      2.284      0.022       0.036       0.469
C(estado_ci

In [40]:
# removendo a variável 'tipo_residencia' poro ter pouca relevância "P>|t|" acima de 5% 

y, X = patsy.dmatrices(formula_like='''
                                    np.log(renda) ~ sexo 
                                                    + posse_de_veiculo 
                                                    + posse_de_imovel 
                                                    + C(qtd_filhos, Treatment(0))
                                                    + C(tipo_renda, Treatment('Assalariado'))
                                                    + C(educacao, Treatment('Secundário')) 
                                                    + C(estado_civil, Treatment('Casado')) 
                                                    + idade 
                                                    + tempo_emprego 
                                                    + C(qt_pessoas_residencia, Treatment(2))
                                    ''', data=df)

sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     232.1
Date:                Sun, 26 May 2024   Prob (F-statistic):               0.00
Time:                        11:38:30   Log-Likelihood:                -13207.
No. Observations:               12090   AIC:                         2.647e+04
Df Residuals:                   12060   BIC:                         2.670e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.0934      0.038    184.940      0.000       7.018       7.169
sexo[T.M]                                                       0.7909      0.015     53.234      0.000       0.762       0.820
posse_de_veiculo[T.True]                                        0.0420      0.014      2.926      0.003       0.014       0.070
posse_de_imovel[T.True]                                         0.0821      0.014      5.886      0.000       0.055       0.109
C(qtd_filhos, Treatment(0))[T.1]                               -0.2483      0.112     -2.220      0.026      -0.468      -0.029
C(qtd_filhos, Treatment(0))[T.2]                               -0.5702      0.223     -2.559      0.011      -1.007      -0.133
C(qtd_filhos, Treatment(0))[T.3]                               -0.5619      0.487     -1.155      0.248      -1.516       0.392
C(qtd_filhos, Treatment(0))[T.4]                                0.1936      0.428      0.452      0.651      -0.646       1.033
C(qtd_filhos, Treatment(0))[T.5]                                0.1173      0.410      0.286      0.775      -0.686       0.921
C(qtd_filhos, Treatment(0))[T.7]                                0.0585      0.181      0.324      0.746      -0.296       0.413
C(qtd_filhos, Treatment(0))[T.14]                              -0.4654      0.262     -1.779      0.075      -0.978       0.047
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2284      0.241      0.946      0.344      -0.245       0.702
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1541      0.015     10.166      0.000       0.124       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.2879      0.243     -1.184      0.236      -0.764       0.189
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0568      0.023      2.523      0.012       0.013       0.101
C(educacao, Treatment('Secundário'))[T.Primário]                0.0096      0.072      0.133      0.894      -0.131       0.150
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1164      0.143      0.816      0.414      -0.163       0.396
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1068      0.014      7.575      0.000       0.079       0.134
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0287      0.033     -0.878      0.380      -0.093       0.035
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3220      0.112      2.866      0.004       0.102       0.542
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2535      0.110      2.297      0.022       0.037       0.470
C(estado_ci


Após remover a variável ***tipo_residencia***, o modelo não apresentou melhora nem piora significativa. O ***R²*** manteve o mesmo valor de **0.358** ou **35,8%** e o **p-value** das variáveis teve uma variação muito pequena, praticamente insignificante.

In [13]:
y, X = patsy.dmatrices(formula_like='''
                                    np.log(renda) ~ sexo 
                                                    + posse_de_veiculo 
                                                    + posse_de_imovel 
                                                    + C(qtd_filhos, Treatment(0))
                                                    + C(tipo_renda, Treatment('Assalariado'))
                                                    + C(educacao, Treatment('Secundário')) 
                                                    + C(estado_civil, Treatment('Casado')) 
                                                    + idade 
                                                    + tempo_emprego
                                    ''', data=df)

sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     279.8
Date:                Sun, 26 May 2024   Prob (F-statistic):               0.00
Time:                        13:55:10   Log-Likelihood:                -13212.
No. Observations:               12090   AIC:                         2.647e+04
Df Residuals:                   12065   BIC:                         2.666e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.0961      0.038    185.722      0.000       7.021       7.171
sexo[T.M]                                                       0.7910      0.015     53.268      0.000       0.762       0.820
posse_de_veiculo[T.True]                                        0.0416      0.014      2.898      0.004       0.013       0.070
posse_de_imovel[T.True]                                         0.0827      0.014      5.925      0.000       0.055       0.110
C(qtd_filhos, Treatment(0))[T.1]                                0.0466      0.017      2.815      0.005       0.014       0.079
C(qtd_filhos, Treatment(0))[T.2]                                0.0828      0.023      3.678      0.000       0.039       0.127
C(qtd_filhos, Treatment(0))[T.3]                               -0.0444      0.055     -0.804      0.421      -0.153       0.064
C(qtd_filhos, Treatment(0))[T.4]                                0.3692      0.176      2.102      0.036       0.025       0.714
C(qtd_filhos, Treatment(0))[T.5]                                0.4118      0.511      0.806      0.420      -0.590       1.414
C(qtd_filhos, Treatment(0))[T.7]                                0.1150      0.362      0.318      0.751      -0.594       0.824
C(qtd_filhos, Treatment(0))[T.14]                              -0.6499      0.512     -1.270      0.204      -1.653       0.353
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2262      0.241      0.937      0.349      -0.247       0.700
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1540      0.015     10.160      0.000       0.124       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3125      0.242     -1.294      0.196      -0.786       0.161
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0576      0.023      2.559      0.010       0.013       0.102
C(educacao, Treatment('Secundário'))[T.Primário]                0.0105      0.072      0.146      0.884      -0.130       0.151
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1122      0.142      0.787      0.431      -0.167       0.391
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1066      0.014      7.565      0.000       0.079       0.134
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0281      0.033     -0.859      0.390      -0.092       0.036
C(estado_civil, Treatment('Casado'))[T.Separado]                0.0390      0.028      1.372      0.170      -0.017       0.095
C(estado_civil, Treatment('Casado'))[T.Solteiro]               -0.0222      0.021     -1.069      0.285      -0.063       0.019
C(estado_ci

In [14]:
smf.ols(formula='''
                np.log(renda) ~ sexo 
                                + posse_de_veiculo 
                                + posse_de_imovel
                                + tipo_renda == 'Empresário'
                                + tipo_renda == 'Servidor público'
                                + educacao == 'Superior completo'
                                + qt_pessoas_residencia == 3
                                + qt_pessoas_residencia == 4
                                + idade 
                                + tempo_emprego
                ''', 
        data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     669.2
Date:                Sun, 26 May 2024   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                -13222.
No. Observations:               12090   AIC:                         2.647e+04
Df Residuals:                   12079   BIC:                         2.655e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    7.0759      0.036    198.812      0.000       7.006       7.146
sexo[T.M]                                    0.7856      0.015     53.240      0.000       0.757       0.815
posse_de_veiculo[T.True]                     0.0420      0.014      2.950      0.003       0.014       0.070
posse_de_imovel[T.True]                      0.0826      0.014      5.937      0.000       0.055       0.110
tipo_renda == 'Empresário'[T.True]           0.1523      0.015     10.078      0.000       0.123       0.182
tipo_renda == 'Servidor público'[T.True]     0.0576      0.023      2.559      0.010       0.013       0.102
educacao == 'Superior completo'[T.True]      0.1084      0.014      7.869      0.000       0.081       0.135
qt_pessoas_residencia == 3[T.True]           0.0481      0.017      2.783      0.005       0.014       0.082
qt_pessoas_residencia == 4[T.True]           0.0904      0.022      4.022      0.000       0.046       0.135
idade                                        0.0061      0.001      7.820      0.000       0.005       0.008
tempo_emprego                                0.0613      0.001     58.284      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.324   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.516   Jarque-Bera (JB):                1.292
Skew:                           0.022   Prob(JB):                        0.524
Kurtosis:                       3.025   Cond. No.                         241.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Comparando o modelo final com o inicial:
Um insight da análise comparativa entre o modelo final e o modelo inicial revela que os indicadores permaneceram praticamente inalterados. O coeficiente de determinação **R²** apresentou uma ligeira redução, passando de 0,358 (35,8%) para 0,357(35,7%), indicando que não houve melhora ou piora significativa no desempenho do modelo.
